# Complete Risk Model Pipeline Test
## Full Functionality Test with GitHub Package Installation

This notebook:
1. Installs the package directly from GitHub (development branch)
2. Creates synthetic test data
3. Tests ALL pipeline functionalities
4. Validates outputs and generates comprehensive reports

**Instructions:**
- Run cells sequentially from top to bottom
- Restart kernel if you encounter import errors

## 1. Install Package from GitHub

In [ ]:
# Install package directly from GitHub development branch
!pip install --upgrade git+https://github.com/selimoksuz/risk-model-pipeline.git@development

# Verify installation
import risk_pipeline
print(f"✅ Package installed successfully!")
print(f"Package location: {risk_pipeline.__file__}")

## 2. Import Required Libraries

In [ ]:
# Standard libraries
import os
import sys
import warnings
import json
import joblib
from datetime import datetime
import time

# Data manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# ML libraries
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import (
    roc_auc_score, classification_report,
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_curve, precision_recall_curve
)
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Try importing XGBoost
try:
    import xgboost as xgb
    HAS_XGBOOST = True
except ImportError:
    HAS_XGBOOST = False
    print("⚠️ XGBoost not installed")

# Settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

# Set random seed
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ Standard libraries imported successfully!")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 3. Import Risk Pipeline Modules

In [ ]:
# Import all modules from risk_pipeline with error handling
modules_status = {}

# Core modules
try:
    from risk_pipeline.core.config import Config
    modules_status['Config'] = '✅'
except ImportError as e:
    modules_status['Config'] = f'❌ {e}'

try:
    from risk_pipeline.core.data_processor import DataProcessor
    modules_status['DataProcessor'] = '✅'
except ImportError as e:
    modules_status['DataProcessor'] = f'❌ {e}'

try:
    from risk_pipeline.core.splitter import DataSplitter
    modules_status['DataSplitter'] = '✅'
except ImportError as e:
    modules_status['DataSplitter'] = f'❌ {e}'

try:
    from risk_pipeline.core.feature_engineer import FeatureEngineer
    modules_status['FeatureEngineer'] = '✅'
except ImportError as e:
    modules_status['FeatureEngineer'] = f'❌ {e}'

try:
    from risk_pipeline.core.feature_selector import FeatureSelector
    modules_status['FeatureSelector'] = '✅'
except ImportError as e:
    modules_status['FeatureSelector'] = f'❌ {e}'

try:
    from risk_pipeline.core.woe_transformer import WOETransformer
    modules_status['WOETransformer'] = '✅'
except ImportError as e:
    modules_status['WOETransformer'] = f'❌ {e}'

try:
    from risk_pipeline.core.model_builder import ModelBuilder
    modules_status['ModelBuilder'] = '✅'
except ImportError as e:
    modules_status['ModelBuilder'] = f'❌ {e}'

try:
    from risk_pipeline.core.model_trainer import ModelTrainer
    modules_status['ModelTrainer'] = '✅'
except ImportError as e:
    modules_status['ModelTrainer'] = f'❌ {e}'

try:
    from risk_pipeline.core.reporter import Reporter
    modules_status['Reporter'] = '✅'
except ImportError as e:
    modules_status['Reporter'] = f'❌ {e}'

try:
    from risk_pipeline.core.report_generator import ReportGenerator
    modules_status['ReportGenerator'] = '✅'
except ImportError as e:
    modules_status['ReportGenerator'] = f'❌ {e}'

try:
    from risk_pipeline.core.psi_calculator import PSICalculator
    modules_status['PSICalculator'] = '✅'
except ImportError as e:
    modules_status['PSICalculator'] = f'❌ {e}'

try:
    from risk_pipeline.core.calibration_analyzer import CalibrationAnalyzer
    modules_status['CalibrationAnalyzer'] = '✅'
except ImportError as e:
    modules_status['CalibrationAnalyzer'] = f'❌ {e}'

try:
    from risk_pipeline.core.risk_band_optimizer import RiskBandOptimizer
    modules_status['RiskBandOptimizer'] = '✅'
except ImportError as e:
    modules_status['RiskBandOptimizer'] = f'❌ {e}'

# Pipeline classes
PIPELINE_CLASS = None
try:
    from risk_pipeline.pipeline import RiskModelPipeline
    PIPELINE_CLASS = RiskModelPipeline
    modules_status['RiskModelPipeline'] = '✅'
except ImportError:
    try:
        from risk_pipeline.complete_pipeline import CompletePipeline
        PIPELINE_CLASS = CompletePipeline
        modules_status['CompletePipeline'] = '✅'
    except ImportError:
        modules_status['Pipeline'] = '❌ No pipeline class available'

# Display import status
print("Module Import Status:")
print("="*40)
for module, status in modules_status.items():
    print(f"{module}: {status}")

# Count successful imports
success_count = sum(1 for s in modules_status.values() if '✅' in str(s))
print(f"\nSuccessfully imported: {success_count}/{len(modules_status)} modules")

## 4. Create Synthetic Test Data

In [ ]:
# Create synthetic dataset
n_samples = 10000
n_features = 30

# Generate classification data
X, y = make_classification(
    n_samples=n_samples,
    n_features=n_features,
    n_informative=20,
    n_redundant=5,
    n_repeated=0,
    n_classes=2,
    n_clusters_per_class=3,
    weights=[0.85, 0.15],  # Imbalanced (15% positive rate)
    flip_y=0.02,  # Add 2% label noise
    random_state=RANDOM_STATE
)

# Create DataFrame
feature_names = [f'feature_{i:02d}' for i in range(n_features)]
df = pd.DataFrame(X, columns=feature_names)
df['target'] = y

# Add categorical features
df['category_1'] = np.random.choice(['A', 'B', 'C', 'D'], size=n_samples)
df['category_2'] = np.random.choice(['Low', 'Medium', 'High'], size=n_samples, p=[0.5, 0.3, 0.2])
df['region'] = np.random.choice(['North', 'South', 'East', 'West', 'Central'], size=n_samples)

# Add some missing values
missing_features = np.random.choice(feature_names[:10], 5, replace=False)
for feat in missing_features:
    missing_idx = np.random.choice(n_samples, int(n_samples * 0.05), replace=False)
    df.loc[missing_idx, feat] = np.nan

# Add ID column
df['customer_id'] = [f'CUST_{i:06d}' for i in range(n_samples)]

# Reorder columns
df = df[['customer_id'] + feature_names + ['category_1', 'category_2', 'region', 'target']]

print(f"✅ Synthetic dataset created!")
print(f"Shape: {df.shape}")
print(f"\nTarget distribution:")
print(df['target'].value_counts())
print(f"Target rate: {df['target'].mean():.2%}")
print(f"\nMissing values:")
missing_summary = df.isnull().sum()
print(missing_summary[missing_summary > 0])
print(f"\nData types:")
print(df.dtypes.value_counts())
print(f"\nFirst 5 rows:")
df.head()

## 5. Configure Pipeline

In [ ]:
# Create configuration
config = Config(
    target_column='target',
    id_column='customer_id',
    test_size=0.2,
    validation_size=0.1,
    random_state=RANDOM_STATE,
    cv_folds=5,
    
    # Feature engineering
    create_polynomial=False,  # Disable for faster testing
    polynomial_degree=2,
    create_interactions=False,
    
    # Feature selection
    selection_method='importance',
    top_k_features=20,
    
    # WOE settings
    max_bins=5,
    min_samples_leaf=0.05,
    
    # Model settings
    scoring_metric='roc_auc',
    
    # Output
    output_folder='test_outputs',
    verbose=True
)

print("✅ Configuration created!")
print(f"\nKey settings:")
print(f"  Target: {config.target_column}")
print(f"  Test size: {config.test_size}")
print(f"  Validation size: {config.validation_size}")
print(f"  Top K features: {config.top_k_features}")
print(f"  Output folder: {config.output_folder}")

## 6. Test Data Processing

In [ ]:
# Test DataProcessor
if 'DataProcessor' in globals():
    processor = DataProcessor(config)
    df_processed = processor.validate_and_freeze(df.copy())
    
    print("✅ Data processing completed!")
    print(f"Processed shape: {df_processed.shape}")
    print(f"\nColumns after processing: {df_processed.shape[1]}")
    
    # Check for missing values
    missing_after = df_processed.isnull().sum().sum()
    print(f"Missing values after processing: {missing_after}")
else:
    print("⚠️ DataProcessor not available")
    df_processed = df.copy()

## 7. Test Data Splitting

In [ ]:
# Test DataSplitter
if 'DataSplitter' in globals():
    splitter = DataSplitter(config)
    splits = splitter.split(df_processed)
    
    print("✅ Data splitting completed!")
    print(f"\nSplit sizes:")
    print(f"  Train: {len(splits['train'])} ({len(splits['train'])/len(df_processed):.1%})")
    print(f"  Validation: {len(splits['validation'])} ({len(splits['validation'])/len(df_processed):.1%})")
    print(f"  Test: {len(splits['test'])} ({len(splits['test'])/len(df_processed):.1%})")
    
    print(f"\nTarget rates:")
    print(f"  Train: {splits['train']['target'].mean():.2%}")
    print(f"  Validation: {splits['validation']['target'].mean():.2%}")
    print(f"  Test: {splits['test']['target'].mean():.2%}")
    
    # Prepare data
    X_train = splits['train'].drop(columns=['target', 'customer_id'])
    y_train = splits['train']['target']
    X_val = splits['validation'].drop(columns=['target', 'customer_id'])
    y_val = splits['validation']['target']
    X_test = splits['test'].drop(columns=['target', 'customer_id'])
    y_test = splits['test']['target']
else:
    print("⚠️ DataSplitter not available, using sklearn")
    # Manual split
    X = df_processed.drop(columns=['target', 'customer_id'])
    y = df_processed['target']
    
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.125, random_state=RANDOM_STATE, stratify=y_temp
    )
    
    print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

## 8. Test Feature Engineering

In [ ]:
# Test FeatureEngineer
if 'FeatureEngineer' in globals():
    engineer = FeatureEngineer(config)
    
    X_train_eng = engineer.create_features(X_train)
    X_val_eng = engineer.transform(X_val)
    X_test_eng = engineer.transform(X_test)
    
    print("✅ Feature engineering completed!")
    print(f"\nFeature counts:")
    print(f"  Original: {X_train.shape[1]}")
    print(f"  After engineering: {X_train_eng.shape[1]}")
    print(f"  New features: {X_train_eng.shape[1] - X_train.shape[1]}")
else:
    print("⚠️ FeatureEngineer not available")
    X_train_eng = X_train
    X_val_eng = X_val
    X_test_eng = X_test

## 9. Test Feature Selection

In [ ]:
# Test FeatureSelector
if 'FeatureSelector' in globals():
    selector = FeatureSelector(config)
    selected_features = selector.select_features(X_train_eng, y_train)
    
    print("✅ Feature selection completed!")
    print(f"\nSelected {len(selected_features)} features from {X_train_eng.shape[1]}")
    
    # Apply selection
    X_train_selected = X_train_eng[selected_features]
    X_val_selected = X_val_eng[selected_features]
    X_test_selected = X_test_eng[selected_features]
    
    print(f"\nTop 10 selected features:")
    print(selected_features[:10].tolist() if hasattr(selected_features, 'tolist') else selected_features[:10])
else:
    print("⚠️ FeatureSelector not available")
    # Select all features
    selected_features = X_train_eng.columns.tolist()
    X_train_selected = X_train_eng
    X_val_selected = X_val_eng
    X_test_selected = X_test_eng

## 10. Test WOE Transformation

In [ ]:
# Test WOETransformer
if 'WOETransformer' in globals():
    woe_transformer = WOETransformer(config)
    
    X_train_woe = woe_transformer.fit_transform(X_train_selected, y_train)
    X_val_woe = woe_transformer.transform(X_val_selected)
    X_test_woe = woe_transformer.transform(X_test_selected)
    
    print("✅ WOE transformation completed!")
    print(f"\nTransformed shape: {X_train_woe.shape}")
    
    # Show sample WOE values
    if hasattr(woe_transformer, 'woe_mapping_') and woe_transformer.woe_mapping_:
        sample_var = list(woe_transformer.woe_mapping_.keys())[0]
        print(f"\nSample WOE mapping for '{sample_var}':")
        print(woe_transformer.woe_mapping_[sample_var].head())
else:
    print("⚠️ WOETransformer not available")
    X_train_woe = X_train_selected
    X_val_woe = X_val_selected
    X_test_woe = X_test_selected

## 11. Test Model Training

In [ ]:
# Train multiple models
models = {
    'logistic_regression': LogisticRegression(random_state=RANDOM_STATE, max_iter=1000),
    'decision_tree': DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=5),
    'random_forest': RandomForestClassifier(n_estimators=100, max_depth=10, random_state=RANDOM_STATE)
}

if HAS_XGBOOST:
    models['xgboost'] = xgb.XGBClassifier(
        n_estimators=100, max_depth=5, 
        random_state=RANDOM_STATE, eval_metric='logloss'
    )

# Train and evaluate models
results = {}
best_model = None
best_score = 0
best_model_name = None

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Train
    model.fit(X_train_woe, y_train)
    
    # Predict
    y_pred_train = model.predict_proba(X_train_woe)[:, 1]
    y_pred_val = model.predict_proba(X_val_woe)[:, 1]
    y_pred_test = model.predict_proba(X_test_woe)[:, 1]
    
    # Calculate scores
    train_score = roc_auc_score(y_train, y_pred_train)
    val_score = roc_auc_score(y_val, y_pred_val)
    test_score = roc_auc_score(y_test, y_pred_test)
    
    results[name] = {
        'model': model,
        'train_score': train_score,
        'val_score': val_score,
        'test_score': test_score,
        'y_pred_test': y_pred_test
    }
    
    print(f"  Train AUC: {train_score:.4f}")
    print(f"  Val AUC: {val_score:.4f}")
    print(f"  Test AUC: {test_score:.4f}")
    
    if val_score > best_score:
        best_score = val_score
        best_model = model
        best_model_name = name

print(f"\n{'='*50}")
print(f"Best Model: {best_model_name} (Val AUC: {best_score:.4f})")
print(f"{'='*50}")

## 12. Model Evaluation

In [ ]:
# Get best model predictions
y_pred_proba = results[best_model_name]['y_pred_test']
y_pred = (y_pred_proba >= 0.5).astype(int)

# Calculate metrics
metrics = {
    'auc': roc_auc_score(y_test, y_pred_proba),
    'gini': 2 * roc_auc_score(y_test, y_pred_proba) - 1,
    'accuracy': accuracy_score(y_test, y_pred),
    'precision': precision_score(y_test, y_pred),
    'recall': recall_score(y_test, y_pred),
    'f1': f1_score(y_test, y_pred)
}

print("Performance Metrics:")
print("="*40)
for metric, value in metrics.items():
    print(f"{metric:10s}: {value:.4f}")

# Confusion Matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(pd.DataFrame(cm, index=['Actual 0', 'Actual 1'], columns=['Pred 0', 'Pred 1']))

# Model comparison
print("\nModel Comparison:")
comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Train AUC': [r['train_score'] for r in results.values()],
    'Val AUC': [r['val_score'] for r in results.values()],
    'Test AUC': [r['test_score'] for r in results.values()],
    'Overfit': [r['train_score'] - r['test_score'] for r in results.values()]
})
print(comparison_df.sort_values('Val AUC', ascending=False))

## 13. Performance Visualization

In [ ]:
# Create performance plots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# ROC Curve
ax = axes[0, 0]
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
ax.plot(fpr, tpr, label=f'AUC = {metrics["auc"]:.3f}')
ax.plot([0, 1], [0, 1], 'k--', alpha=0.5)
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curve')
ax.legend()
ax.grid(True, alpha=0.3)

# Score Distribution
ax = axes[0, 1]
ax.hist(y_pred_proba[y_test == 0], bins=30, alpha=0.5, label='Class 0', color='blue')
ax.hist(y_pred_proba[y_test == 1], bins=30, alpha=0.5, label='Class 1', color='red')
ax.set_xlabel('Predicted Probability')
ax.set_ylabel('Frequency')
ax.set_title('Score Distribution')
ax.legend()
ax.grid(True, alpha=0.3)

# Confusion Matrix Heatmap
ax = axes[1, 0]
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title('Confusion Matrix')

# Precision-Recall Curve
ax = axes[1, 1]
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
ax.plot(recall, precision)
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_title('Precision-Recall Curve')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Performance plots generated!")

## 14. Test PSI Calculator

In [ ]:
# Test PSICalculator
if 'PSICalculator' in globals():
    psi_calculator = PSICalculator()
    
    # Calculate score PSI
    y_train_pred = best_model.predict_proba(X_train_woe)[:, 1]
    score_psi = psi_calculator.calculate(y_train_pred, y_pred_proba)
    
    print("✅ PSI Analysis completed!")
    print(f"\nScore PSI (Train vs Test): {score_psi:.4f}")
    
    # Interpretation
    if score_psi < 0.1:
        print("  ✅ Model is stable (PSI < 0.1)")
    elif score_psi < 0.25:
        print("  ⚠️ Minor shift detected (0.1 <= PSI < 0.25)")
    else:
        print("  ❌ Significant shift detected (PSI >= 0.25)")
    
    # Feature PSI for top features
    print("\nFeature PSI (Top 5 features):")
    for i, col in enumerate(X_train_woe.columns[:5]):
        feature_psi = psi_calculator.calculate(X_train_woe[col], X_test_woe[col])
        status = "✅" if feature_psi < 0.1 else "⚠️" if feature_psi < 0.25 else "❌"
        print(f"  {col}: {feature_psi:.4f} {status}")
else:
    print("⚠️ PSICalculator not available")
    score_psi = 0.0

## 15. Test Calibration Analysis

In [ ]:
# Test CalibrationAnalyzer
if 'CalibrationAnalyzer' in globals():
    calibration_analyzer = CalibrationAnalyzer()
    
    # Analyze calibration
    cal_results = calibration_analyzer.analyze_calibration(y_test, y_pred_proba)
    
    print("✅ Calibration analysis completed!")
    print("\nCalibration Metrics:")
    print(f"  Expected Calibration Error (ECE): {cal_results['ece']:.4f}")
    print(f"  Maximum Calibration Error (MCE): {cal_results['mce']:.4f}")
    print(f"  Brier Score: {cal_results['brier_score']:.4f}")
    
    # Interpretation
    if cal_results['ece'] < 0.05:
        print("\n✅ Model is well calibrated (ECE < 0.05)")
    elif cal_results['ece'] < 0.1:
        print("\n⚠️ Model has minor calibration issues (0.05 <= ECE < 0.1)")
    else:
        print("\n❌ Model needs calibration (ECE >= 0.1)")
    
    # Calibration plot
    if hasattr(calibration_analyzer, 'plot_calibration'):
        try:
            fig = calibration_analyzer.plot_calibration(y_test, y_pred_proba)
            plt.show()
        except:
            print("Could not generate calibration plot")
else:
    print("⚠️ CalibrationAnalyzer not available")
    cal_results = {'ece': 0.0, 'mce': 0.0, 'brier_score': 0.0}

## 16. Test Risk Band Optimizer

In [ ]:
# Test RiskBandOptimizer
if 'RiskBandOptimizer' in globals():
    risk_band_optimizer = RiskBandOptimizer()
    
    # Create risk bands
    risk_bands = risk_band_optimizer.optimize_bands(
        y_true=y_test,
        y_scores=y_pred_proba,
        n_bands=5,
        method='quantile'
    )
    
    print("✅ Risk band optimization completed!")
    print("\nRisk Bands:")
    print(risk_bands[['band', 'min_score', 'max_score', 'count', 'bad_rate', 'volume_pct']])
    
    # Check monotonicity
    is_monotonic = all(risk_bands['bad_rate'].iloc[i] <= risk_bands['bad_rate'].iloc[i+1] 
                      for i in range(len(risk_bands)-1))
    print(f"\nRisk bands are {'✅ monotonic' if is_monotonic else '❌ not monotonic'}")
    
    # Visualize risk bands
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Bad rate by band
    ax = axes[0]
    ax.bar(risk_bands['band'], risk_bands['bad_rate'], color='coral')
    ax.set_xlabel('Risk Band')
    ax.set_ylabel('Bad Rate')
    ax.set_title('Bad Rate by Risk Band')
    ax.grid(True, alpha=0.3)
    
    # Volume distribution
    ax = axes[1]
    ax.bar(risk_bands['band'], risk_bands['volume_pct'], color='skyblue')
    ax.set_xlabel('Risk Band')
    ax.set_ylabel('Volume %')
    ax.set_title('Volume Distribution')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ RiskBandOptimizer not available")
    risk_bands = pd.DataFrame()

## 17. Test Complete Pipeline

In [ ]:
# Test complete pipeline if available
if PIPELINE_CLASS:
    print(f"Testing {PIPELINE_CLASS.__name__}...\n")
    
    # Create fresh dataset
    X_pipe, y_pipe = make_classification(
        n_samples=5000, n_features=25, n_informative=18,
        n_redundant=5, n_classes=2, weights=[0.8, 0.2],
        random_state=RANDOM_STATE+1
    )
    
    df_pipeline = pd.DataFrame(X_pipe, columns=[f'var_{i:02d}' for i in range(X_pipe.shape[1])])
    df_pipeline['target'] = y_pipe
    
    # Initialize pipeline
    pipeline = PIPELINE_CLASS(config)
    
    try:
        # Fit pipeline
        pipeline.fit(df_pipeline)
        
        # Get predictions
        predictions = pipeline.predict(df_pipeline)
        probabilities = pipeline.predict_proba(df_pipeline)
        
        # Evaluate
        pipeline_score = roc_auc_score(y_pipe, probabilities[:, 1])
        
        print(f"✅ Pipeline test successful!")
        print(f"Pipeline AUC: {pipeline_score:.4f}")
        
        # Save pipeline
        os.makedirs(config.output_folder, exist_ok=True)
        pipeline_path = os.path.join(config.output_folder, 'complete_pipeline.pkl')
        joblib.dump(pipeline, pipeline_path)
        print(f"Pipeline saved to: {pipeline_path}")
        
    except Exception as e:
        print(f"⚠️ Pipeline test failed: {e}")
else:
    print("⚠️ No pipeline class available")

## 18. Final Summary

In [ ]:
print("="*70)
print("COMPLETE PIPELINE TEST SUMMARY")
print("="*70)
print(f"\n📦 Package: risk-model-pipeline (development branch)")
print(f"⏰ Test Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print("\n✅ MODULES TESTED:")
for module, status in modules_status.items():
    if '✅' in str(status):
        print(f"  ✓ {module}")

print("\n❌ MODULES NOT AVAILABLE:")
for module, status in modules_status.items():
    if '❌' in str(status):
        print(f"  ✗ {module}")

print("\n📊 BEST MODEL RESULTS:")
print(f"  Model: {best_model_name}")
print(f"  Test AUC: {results[best_model_name]['test_score']:.4f}")
print(f"  Gini: {metrics['gini']:.4f}")
print(f"  Accuracy: {metrics['accuracy']:.4f}")

if 'PSICalculator' in globals():
    print(f"\n📈 STABILITY METRICS:")
    print(f"  PSI: {score_psi:.4f}")

if 'CalibrationAnalyzer' in globals():
    print(f"  ECE: {cal_results['ece']:.4f}")

if 'RiskBandOptimizer' in globals() and not risk_bands.empty:
    print(f"  Risk Bands: {len(risk_bands)} bands")

print("\n" + "="*70)
print("🎉 TEST COMPLETED SUCCESSFULLY! 🎉")
print("="*70)